In [11]:
import json
import os
import requests
import time
import pickle
import gzip
import re

# Found dataset of NYT Facebook posts

From https://data.world/martinchek/2012-2016-facebook-posts

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ml_tools import eda, nlp_prep

In [19]:
# Sublime Text told me the encoding was UTF-16 LE with BOM
df = pd.read_csv('data/the-new-york-times-5281959998.csv', encoding='utf_16_le', 
                 engine='python')
df.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at
0,5281959998_416259568436280,5281959998,Did Barack Obama Save Ohio?,"'There are several pivotal states — Ohio, Flor...",Why the battle to take credit for Ohio’s ever-...,nytimes.com,link,shared_story,413,44,68,0,0,0,0,0,0,http://www.nytimes.com/2012/09/09/magazine/ohi...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 15:16:55
1,5281959998_268320586619940,5281959998,The Weatherman Is Not a Moron,"In 1940, the chance of an American being kille...",In the hocus-pocus realm of predicting the fut...,nytimes.com,link,shared_story,338,14,78,0,0,0,0,0,0,http://www.nytimes.com/2012/09/09/magazine/the...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:01:13
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,421,0,0,0,0,0,0,http://www.nytimes.com/2012/09/07/opinion/roge...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:44:02
3,5281959998_10150252974589999,5281959998,Timeline Photos,Our special Arts & Leisure section 'The New Se...,NaN,NaN,photo,added_photos,513,17,99,0,0,0,0,0,0,https://www.facebook.com/nytimes/photos/a.2835...,https://scontent.xx.fbcdn.net/v/t1.0-0/s130x13...,2012-09-08 17:32:18
4,5281959998_463550620334374,5281959998,Pot for Parents,'When I’m rolling around the floor with my gig...,"I am a more loving, attentive and patient fath...",nytimes.com,link,shared_story,502,83,170,0,0,0,0,0,0,http://www.nytimes.com/2012/09/08/opinion/how-...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 19:14:51


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47867 entries, 0 to 47866
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              47867 non-null  object
 1   page_id         47867 non-null  int64 
 2   name            46574 non-null  object
 3   message         46132 non-null  object
 4   description     43512 non-null  object
 5   caption         42892 non-null  object
 6   post_type       47867 non-null  object
 7   status_type     47856 non-null  object
 8   likes_count     47867 non-null  int64 
 9   comments_count  47867 non-null  int64 
 10  shares_count    47867 non-null  int64 
 11  love_count      47867 non-null  int64 
 12  wow_count       47867 non-null  int64 
 13  haha_count      47867 non-null  int64 
 14  sad_count       47867 non-null  int64 
 15  thankful_count  47867 non-null  int64 
 16  angry_count     47867 non-null  int64 
 17  link            47201 non-null  object
 18  pictur

In [7]:
df.describe()

,page_id,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count
count,4.786700e+04,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000
mean,5.281960e+09,1883.853761,179.163599,503.113690,38.096288,14.206301,15.359851,28.534084,0.032653,28.756429
std,0.000000e+00,6223.729785,998.989321,3258.821754,452.819483,151.503261,151.798665,318.864824,1.792674,357.463338
min,5.281960e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.281960e+09,335.000000,33.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.281960e+09,697.000000,73.000000,147.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.281960e+09,1620.000000,173.000000,404.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,5.281960e+09,497519.000000,169805.000000,528856.000000,56070.000000,21121.000000,13214.000000,32316.000000,255.000000,48578.000000


# NYT Initial Articles Matching

In [21]:
f = gzip.open('data/nyt_articles_2012_to_2016.pickle.gz','rb')
articles = pickle.load(f)
f.close()

df_nyt = pd.DataFrame(articles, columns=articles[0].keys())
df_nyt.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
0,The best New Year’s Day news for Chicago’s spo...,https://www.nytimes.com/2012/01/01/sports/look...,The best New Year’s Day news for Chicago’s spo...,Dan McGrath writes a column for The Chicago Ne...,A,25B,The New York Times,"[{'rank': 0, 'subtype': 'jumbo', 'caption': No...",{'main': 'A Year Full of Missteps Is Finally O...,"[{'name': 'organizations', 'value': 'Chicago C...",2012-01-01T00:00:28+0000,article,National,Sports,"{'original': 'By Dan McGrath', 'person': [{'fi...",News,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...,912,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...
1,The man was preparing to board a flight in Mid...,https://www.nytimes.com/2012/01/01/us/servicem...,The man was preparing to board a flight in Mid...,HOUSTON — A man preparing to board a flight at...,A,19,The New York Times,[],{'main': 'Serviceman Held for Carrying Explosi...,"[{'name': 'subject', 'value': 'Airport Securit...",2012-01-01T00:09:12+0000,article,National,U.S.,"{'original': 'By Manny Fernandez', 'person': [...",News,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...,242,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...
2,"Setting the scene, Hollywood style, for what c...",https://fifthdown.blogs.nytimes.com/2011/12/31...,"Setting the scene, Hollywood style, for what c...","Yes, the Jets are a long shot to qualify for t...",NaN,NaN,The New York Times,[],{'main': 'Picturing How Ryan Brothers Could Se...,"[{'name': 'subject', 'value': 'Football', 'ran...",2012-01-01T00:14:59+0000,article,,Sports,"{'original': 'By George Bretherton', 'person':...",News,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...,491,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...
3,Experts said the broadcasts were most likely p...,https://thecaucus.blogs.nytimes.com/2011/12/31...,Experts said the broadcasts were most likely p...,DES MOINES — Can they do that?,NaN,NaN,The New York Times,[],{'main': 'Experts Weigh In on Outside Groups’ ...,"[{'name': 'subject', 'value': 'Campaign Financ...",2012-01-01T00:15:52+0000,article,,U.S.,"{'original': 'By Nicholas Confessore', 'person...",News,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...,449,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...
4,After more than a decade in which San Francisc...,https://www.nytimes.com/2012/01/01/us/an-unlik...,After more than a decade in which San Francisc...,"Susan Beckstead stepped out of her sky-blue, t...",A,25A,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'An Unlikely Group Rebels Against Pre...,"[{'name': 'subject', 'value': 'Historic Buildi...",2012-01-01T00:15:54+0000,article,National,U.S.,"{'original': 'By Matt Smith', 'person': [{'fir...",News,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...,1164,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...


In [22]:
df_nyt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425931 entries, 0 to 425930
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   abstract          425931 non-null  object
 1   web_url           425931 non-null  object
 2   snippet           425931 non-null  object
 3   lead_paragraph    425931 non-null  object
 4   print_section     243058 non-null  object
 5   print_page        243071 non-null  object
 6   source            425931 non-null  object
 7   multimedia        425931 non-null  object
 8   headline          425931 non-null  object
 9   keywords          425931 non-null  object
 10  pub_date          425931 non-null  object
 11  document_type     425931 non-null  object
 12  news_desk         425931 non-null  object
 13  section_name      425931 non-null  object
 14  byline            425931 non-null  object
 15  type_of_material  425931 non-null  object
 16  _id               425931 non-null  obj

## Initial preprocessing for matching

### Pull Links from messages

In [23]:
df.loc[(df['link'].isna()==False) & (
       (df['link'].str.contains('nytimes.com')) |
       (df['link'].str.contains('nyti.ms'))), 'has_nyt_link'] = 1

In [27]:
df.loc[(df['link'].isna()==True) |
       (df['has_nyt_link'].isna()), ['post_type', 'link', 'message']][:5].values

array([['photo',
        'https://www.facebook.com/nytimes/photos/a.283559809998.33779.5281959998/10150252974559999/?type=3',
        "Our special Arts & Leisure section 'The New Season' is now online. Take a look at the upcoming offerings in arts and entertainment below. Are you looking forward to any particular events?Art: http://nyti.ms/TAYvoJClassical: http://nyti.ms/P41kvVDance: http://nyti.ms/Rx3YIhMovies: http://nyti.ms/Ow4dodPop: http://nyti.ms/RoEIJUTelevision: http://nyti.ms/Rx4fuRTheater: http://nyti.ms/OhVSZA(Credit: Jonathan Bartlett)"],
       ['photo',
        'https://www.facebook.com/nytimes/photos/a.283559809998.33779.5281959998/10150253031054999/?type=3',
        'During a surprise visit to Gators Dockside bar in Orlando, Florida, President Obama was introduced to a boy who was born in Hawaii. The president lit up and flashed the Hawaiian shaka hand sign, which the boy — Andre Wupperman, who will be 7 next week — returned. “You have a birth certificate?” the presiden

In [40]:
df_messages = df.loc[((df['link'].isna()) | (df['has_nyt_link'].isna())) 
                     & (df['message'].isna()==False), 
                     ['id', 'post_type', 'message', 'description']]

In [41]:
# pull shortened links out
df_messages = nlp_prep.pattern_match_in_df(df_messages, doc_col='message', 
                hit_col='message_link1', 
                pattern=r"(http[s]?://nyti.ms/[A-Za-z0-9/.]+)", 
                out_type='list', replace=False)

In [43]:
# pull longer links out
df_messages = nlp_prep.pattern_match_in_df(df_messages, doc_col='message', 
                hit_col='message_link2', 
                pattern=r"(http[s]?://nytimes/[A-Za-z0-9/.-_]+)", 
                out_type='list', replace=False)

In [50]:
df_messages.loc[df_messages['message_link1'].str.len() > 0]

,id,post_type,message,description,message_link1,message_link2
3,5281959998_10150252974589999,photo,Our special Arts & Leisure section 'The New Se...,NaN,"[http://nyti.ms/Rx3YIhMovies, http://nyti.ms/R...",[]
5,5281959998_10150253031069999,photo,During a surprise visit to Gators Dockside bar...,NaN,[http://nyti.ms/NTb67a],[]
13,5281959998_10150253189039999,photo,President Obama got a huge lift—literally — fr...,NaN,[http://nyti.ms/PUEe9T],[]
14,5281959998_10150253207659999,photo,Serena Williams defeated Victoria Azarenka to ...,NaN,[http://nyti.ms/P8LIFN],[]
22,5281959998_10150253392344999,photo,What's on your menu for lunch or dinner today?...,NaN,[http://nyti.ms/RwXkrk],[]
...,...,...,...,...,...,...
47787,5281959998_1301608653216932,video,If you haven’t yet figured out a go-to recipe ...,NaN,[http://nyti.ms/2fpKB8d],[]
47792,5281959998_10150956594114999,photo,Ronald Reagan ended up winning 49 states in a ...,NaN,[http://nyti.ms/2fIAhNx],[]
47800,5281959998_10150956824229999,photo,"Read the letter that James Comey, the FBI dire...",NaN,[http://nyti.ms/2fpV8Se],[]
47815,5281959998_10150957229929999,photo,Election week is here. Enjoy unlimited access ...,NaN,[http://nyti.ms/2fPBG4g],[]


OK so only link1 got populated; the messages didn't ever include full nytimes links. Let's pull out the first one.

In [52]:
df_messages['message_link'] = df_messages['message_link1'].map(
    lambda x: x[0] if len(x) > 0 else '')

In [53]:
df_messages.loc[df_messages['message_link'] != '']

,id,post_type,message,description,message_link1,message_link2,message_link
3,5281959998_10150252974589999,photo,Our special Arts & Leisure section 'The New Se...,NaN,"[http://nyti.ms/Rx3YIhMovies, http://nyti.ms/R...",[],http://nyti.ms/Rx3YIhMovies
5,5281959998_10150253031069999,photo,During a surprise visit to Gators Dockside bar...,NaN,[http://nyti.ms/NTb67a],[],http://nyti.ms/NTb67a
13,5281959998_10150253189039999,photo,President Obama got a huge lift—literally — fr...,NaN,[http://nyti.ms/PUEe9T],[],http://nyti.ms/PUEe9T
14,5281959998_10150253207659999,photo,Serena Williams defeated Victoria Azarenka to ...,NaN,[http://nyti.ms/P8LIFN],[],http://nyti.ms/P8LIFN
22,5281959998_10150253392344999,photo,What's on your menu for lunch or dinner today?...,NaN,[http://nyti.ms/RwXkrk],[],http://nyti.ms/RwXkrk
...,...,...,...,...,...,...,...
47787,5281959998_1301608653216932,video,If you haven’t yet figured out a go-to recipe ...,NaN,[http://nyti.ms/2fpKB8d],[],http://nyti.ms/2fpKB8d
47792,5281959998_10150956594114999,photo,Ronald Reagan ended up winning 49 states in a ...,NaN,[http://nyti.ms/2fIAhNx],[],http://nyti.ms/2fIAhNx
47800,5281959998_10150956824229999,photo,"Read the letter that James Comey, the FBI dire...",NaN,[http://nyti.ms/2fpV8Se],[],http://nyti.ms/2fpV8Se
47815,5281959998_10150957229929999,photo,Election week is here. Enjoy unlimited access ...,NaN,[http://nyti.ms/2fPBG4g],[],http://nyti.ms/2fPBG4g


### Expand links pulled from messages

I'm going to append the id instead of the index, since it's less likely to change.

In [54]:
# use urllib to expand URLs, since some are shortened
# code adapted from:
# https://stackoverflow.com/questions/4201062/how-can-i-unshorten-a-url
expanded_link2 = []
session2 = requests.Session()  # so connections are recycled
count2 = 0

for i in df_messages.loc[df_messages['message_link'] != ''].index:
    try:
        resp2 = session2.head(df_messages.at[i, 'message_link'], 
                            allow_redirects=True)
        expanded_link2.append([df_messages.at[i, 'id'], resp2.url])
    except:
        print(f"Error at index {i}")
    
    count2 += 1
    
    if count2 % 500 == 0:
        print(f"{count2} links processed")
        
expanded_link2[:5]

Error at index 1989
Error at index 2266
Error at index 2913
500 links processed
1000 links processed
1500 links processed
2000 links processed
2500 links processed
3000 links processed


[['5281959998_10150252974589999', 'https://www.nytimes.com/'],
 ['5281959998_10150253031069999',
  'https://thecaucus.blogs.nytimes.com/2012/09/08/obama-makes-birth-certificate-joke/?smid=FB-nytimes&WT.mc_id=PO-E-FB-SM-PIX-OMB-090812-NYT-NA&WT.mc_ev=click'],
 ['5281959998_10150253189039999',
  'https://thecaucus.blogs.nytimes.com/2012/09/09/obama-gets-a-lift-on-the-trail/?smid=FB-nytimes&WT.mc_id=PO-E-FB-SM-PIX-OGL-090912-NYT-NA&WT.mc_ev=click'],
 ['5281959998_10150253207659999',
  'https://archive.nytimes.com/www.nytimes.com/interactive/2012/09/08/sports/tennis/usopen-womens-final-live.html?WT.mc_ev=click&WT.mc_id=SP-E-FB-SM-PIX-LUF-090912-NYT-NA&smid=FB-nytimes'],
 ['5281959998_10150253392344999',
  'https://well.blogs.nytimes.com/2012/09/07/wholesome-risotto-with-whole-grains/?smid=FB-nytimes&WT.mc_id=HL-E-FB-SM-PIX-WRW-091012-NYT-NA&WT.mc_ev=click']]

In [56]:
df_expanded2 = pd.DataFrame(expanded_link2, columns=['id', 'link_expanded'])
df_expanded2.tail()

,id,link_expanded
3131,5281959998_1301608653216932,http://nyti.ms/2fpKB8d
3132,5281959998_10150956594114999,http://nyti.ms/2fIAhNx
3133,5281959998_10150956824229999,http://nyti.ms/2fpV8Se
3134,5281959998_10150957229929999,http://nyti.ms/2fPBG4g
3135,5281959998_10150957696529999,http://nyti.ms/2eO28Hv


In [58]:
df_expanded2.loc[df_expanded2['link_expanded'].str.contains('http://nyti.ms/', 
                                                regex=False)]

,id,link_expanded
3076,5281959998_10150919238784999,http://nyti.ms/2dIcXPs
3077,5281959998_1265081713536293,http://nyti.ms/2duWevr
3078,5281959998_10150920129559999,http://nyti.ms/2dFWD0ePhoto
3079,5281959998_10150920525779999,http://nyti.ms/2dUoB60Photo
3080,5281959998_10150921036729999,http://nyti.ms/2dOqfWu
3081,5281959998_10150921534584999,http://nyti.ms/2dAaun7
3082,5281959998_10150921718124999,http://nyti.ms/2dZWmGB
3083,5281959998_10150921836114999,http://nyti.ms/2dQYWuu
3084,5281959998_1764525223764992,http://nyti.ms/2d2daac
3085,5281959998_1267676913276773,http://nyti.ms/1Jy31lc


In [59]:
# use urllib to expand URLs, since some are shortened
# code adapted from:
# https://stackoverflow.com/questions/4201062/how-can-i-unshorten-a-url

session2 = requests.Session()  # so connections are recycled
count2 = 0

for i in df_expanded2.loc[df_expanded2['link_expanded']\
                    .str.contains('http://nyti.ms/', regex=False)].index:
    try:
        resp2 = session2.head(df_expanded2.at[i, 'link_expanded'], 
                            allow_redirects=True)
        df_expanded2.at[i, 'link_expanded'] = resp2.url
    except:
        print(f"Error at index {i}")
    
    count2 += 1
    
    if count2 % 500 == 0:
        print(f"{count2} links processed")
        

In [63]:
df_expanded2.loc[df_expanded2['link_expanded']=='https://www.nytimes.com/']

,id,link_expanded
0,5281959998_10150252974589999,https://www.nytimes.com/
6,5281959998_10150253437759999,https://www.nytimes.com/
9,5281959998_10150253545774999,https://www.nytimes.com/
10,5281959998_10150253622359999,https://www.nytimes.com/
15,5281959998_10150254204124999,https://www.nytimes.com/
...,...,...
3115,5281959998_10150945228609999,https://www.nytimes.com/
3116,5281959998_10150945225049999,https://www.nytimes.com/
3123,5281959998_10150949958844999,https://www.nytimes.com/
3124,5281959998_10150950007859999,https://www.nytimes.com/


In [64]:
# Links still not updated
df_expanded2.drop(df_expanded2.loc[df_expanded2['link_expanded']==
                                   'https://www.nytimes.com/'].index, inplace=True)

In [67]:
# export saved expanded links that we have
df_expanded2.loc[df_expanded2['link_expanded'].isna()==False, 
                 ['id', 'link_expanded']
                ].to_json('data/expanded_links_frommessages.json', )

In [12]:
# replace http with https, since https is used in nytimes links
df['trim_link'] = df['link'].map(lambda x: str.replace(x, 'http:', 'https:') if
                             type(x)==str else '')

In [13]:
# pull out the page link, without any extra & and ?
df['trim_link'] = df['trim_link'].map(
    lambda x: re.search(r"^([^?&]+)", x).group(1) 
                if (type(x)==str and re.search(r"^([^?&]+)", x)) else '')

In [14]:
df['status_type'].value_counts()

shared_story            43359
added_photos             2249
added_video              1574
mobile_status_update      654
created_note               11
created_event               9
Name: status_type, dtype: int64

In [15]:
# drop facebook updates for updates other than sharing stories
df.drop(index=df[df['status_type']!='shared_story'].index, inplace=True)

In [16]:
df['status_type'].value_counts()

shared_story    43359
Name: status_type, dtype: int64

In [17]:
# Review duplicates where we have the same link, description, and status_type
df[df.duplicated(subset=['description', 'trim_link', 'status_type'], 
                 keep=False)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at,trim_link
277,5281959998_480453791987573,5281959998,"Unfriending Someone, Before Facebook","When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...",nytimes.com,link,shared_story,454,41,...,0,0,0,0,0,0,http://www.nytimes.com/2012/10/03/booming/03wa...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-03 16:10:40,https://www.nytimes.com/2012/10/03/booming/03w...
291,5281959998_429781297079467,5281959998,"Unfriending Someone, Before Facebook",Is unfriending a person on Facebook the right ...,"How much better things were 50 years ago, when...",nytimes.com,link,shared_story,461,69,...,0,0,0,0,0,0,http://www.nytimes.com/2012/10/03/booming/03wa...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-04 20:17:42,https://www.nytimes.com/2012/10/03/booming/03w...
397,5281959998_123566444461425,5281959998,Presidential Debate Fact-Checks and Updates - ...,Follow our live coverage for rapid NYT fact-ch...,President Obama and Mitt Romney square off on ...,elections.nytimes.com,link,shared_story,215,35,...,0,0,0,0,0,0,http://elections.nytimes.com/2012/debates/pres...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-17 01:08:07,https://elections.nytimes.com/2012/debates/pre...
398,5281959998_532464846767481,5281959998,Presidential Debate Fact-Checks and Updates - ...,What do you think of the debate so far? Follow...,President Obama and Mitt Romney square off on ...,elections.nytimes.com,link,shared_story,409,274,...,0,0,0,0,0,0,http://elections.nytimes.com/2012/debates/pres...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-17 01:26:53,https://elections.nytimes.com/2012/debates/pre...
504,5281959998_508352792509259,5281959998,U.S. Economy Grew at 2% Rate in 3rd Quarter,The U.S. economy grew at an annual rate of 2 p...,More positive consumer activity and a healthie...,nytimes.com,link,shared_story,730,72,...,0,0,0,0,0,0,http://www.nytimes.com/2012/10/27/business/eco...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-26 12:53:54,https://www.nytimes.com/2012/10/27/business/ec...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47641,5281959998_10150952237769999,5281959998,"Hillary Clinton Still Leads a Tighter Race, Ti...",Breaking News: Hillary Clinton holds a 3-point...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,6102,634,...,715,31,82,9,0,28,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-03 11:03:02,https://www.nytimes.com/2016/11/04/us/politics...
47704,5281959998_10150953685229999,5281959998,Voters Express Disgust Over U.S. Politics in N...,A majority of voters say that neither Hillary ...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,474,182,...,3,10,5,52,0,4,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 14:55:00,https://www.nytimes.com/2016/11/04/us/politics...
47723,5281959998_10150954236134999,5281959998,2016 Election Forecast: Who Will Be President?,"With days to go, here's our presidential forec...","The Upshot’s presidential forecast, updated da...",nytimes.com,link,shared_story,2904,390,...,426,19,141,5,0,13,http://www.nytimes.com/interactive/2016/upshot...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 22:55:00,https://www.nytimes.com/interactive/2016/upsho...
47727,5281959998_10150954325034999,5281959998,Cranberry Curd Tart Recipe,What a beauty. (via The New York Times Food),If you are a fan of lemon curd or the classic ...,cooking.nytimes.com,link,shared_story,3288,282,...,268,160,4,1,0,1,http://cooking.nytimes.com/recipes/1017817-cra...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-05 00:55:00,https

In [18]:
df.loc[[47641, 47704]]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at,trim_link
47641,5281959998_10150952237769999,5281959998,"Hillary Clinton Still Leads a Tighter Race, Ti...",Breaking News: Hillary Clinton holds a 3-point...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,6102,634,...,715,31,82,9,0,28,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-03 11:03:02,https://www.nytimes.com/2016/11/04/us/politics...
47704,5281959998_10150953685229999,5281959998,Voters Express Disgust Over U.S. Politics in N...,A majority of voters say that neither Hillary ...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,474,182,...,3,10,5,52,0,4,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 14:55:00,https://www.nytimes.com/2016/11/04/us/politics...


In [19]:
ex_desc = df.loc[[47641]].values
ex_desc

array([['5281959998_10150952237769999', 5281959998,
        'Hillary Clinton Still Leads a Tighter Race, Times/CBS News Poll Shows',
        'Breaking News: Hillary Clinton holds a 3-point lead over Donald J. Trump in the latest Times/CBS News poll. Most voters say their minds are made up.',
        'Most voters say that their minds are made up and that late revelations about Mrs. Clinton and Donald J. Trump made no significant difference to them.',
        'nytimes.com', 'link', 'shared_story', 6102, 634, 628, 715, 31,
        82, 9, 0, 28,
        'http://www.nytimes.com/2016/11/04/us/politics/hillary-clinton-donald-trump-poll.html?smid=fb-nytimes&smtyp=cur',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQBMxkLxy577UjpI&w=130&h=130&url=https%3A%2F%2Fstatic01.nyt.com%2Fimages%2F2016%2F11%2F03%2Fus%2F04poll1%2F04poll1-facebookJumbo.jpg&cfs=1&sx=224&sy=0&sw=550&sh=550',
        '2016-11-03 11:03:02',
        'https://www.nytimes.com/2016/11/04/us/politics/hillary-clinton-dona

I'm not sure I can just drop these completely... It looks like maybe the duplicates could represent some experimentation with framing the story differently to capture the attention of audiences better, and it's possible this resulted in significantly different numbers of comments and likes. It's also possible that the change is due to going with an initial headline for breaking news, and changing it later. The first example was a breaking news story at first, and it was also related to results of a poll, from which many conclusions could be drawn. Maybe they just needed more time to tease additional insights out of the story.

What I would like to do is mark these duplicates so I can try to match the API articles based on just one copy, and then I will need to go back and take a look at the details to decide which version should actually be kept. 

Perhaps both, since they will have different message values and different numbers of comments and likes, so we could do a direct comparison to see which did better. Maybe the second post could be in the test set?

In [21]:
# populate 'dupes_on' column for all copies indicating how duplicate status
# was determined. `link` means these are duplicates based on the link
df.loc[df.duplicated(subset=['trim_link', 'status_type'], keep=False), 
       'dupes_on_link'] = 1

# populate `dupe_child` column for all but the first occurences
df.loc[df.duplicated(subset=['trim_link', 'status_type'], keep='first'), 
       'dupe_link_child'] = 1

In [22]:
# identify duplicates based on description only, since I want to try to
# use that to join with nyt articles
df.loc[(df.duplicated(subset=['description'], keep=False)) &
        (df['description'].isna()==False), 'dupes_on_desc'] = 1

df.loc[(df.duplicated(subset=['description'], keep='first')) &
       (df['description'].isna()==False), 'dupe_desc_child'] = 1

In [23]:
# identify duplicates based on name
df.loc[(df.duplicated(subset=['name'], keep=False)) &
        (df['name'].isna()==False), 'dupes_on_name'] = 1

df.loc[(df.duplicated(subset=['name'], keep='first')) &
       (df['name'].isna()==False), 'dupe_name_child'] = 1

In [24]:
df.loc[(df['dupes_on_link']==1) | 
       (df['dupes_on_desc']==1) | 
       (df['dupes_on_name']==1)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,link,picture,posted_at,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child
20,5281959998_353360298082368,5281959998,"Teacher Strike Begins in Chicago, Amid Signs T...","Teachers in Chicago Public Schools, the nation...","As about 350,000 students got an unexpected da...",nytimes.com,link,shared_story,187,19,...,http://www.nytimes.com/2012/09/11/education/te...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-10 15:36:47,https://www.nytimes.com/2012/09/11/education/t...,1.0,NaN,NaN,NaN,NaN,NaN
31,5281959998_179738532162181,5281959998,Teachers’ Strike in Chicago Tests Mayor and Union,Quotation of the Day: 'Don’t take it out on th...,"The strike left 350,000 children without class...",nytimes.com,link,shared_story,230,36,...,http://www.nytimes.com/2012/09/11/education/te...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-11 12:05:55,https://www.nytimes.com/2012/09/11/education/t...,1.0,1.0,NaN,NaN,NaN,NaN
45,5281959998_137004556446145,5281959998,U.S. Envoy to Libya Is Killed in Attack,"The United States ambassador to Libya, J. Chri...",The American ambassador Christopher Stevens wa...,nytimes.com,link,shared_story,337,228,...,http://www.nytimes.com/2012/09/13/world/middle...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-12 12:53:53,https://www.nytimes.com/2012/09/13/world/middl...,1.0,NaN,NaN,NaN,1.0,NaN
49,5281959998_205704876226189,5281959998,U.S. Envoy to Libya Is Killed in Attack,“These four Americans stood up for freedom and...,Ambassador Christopher Stevens was killed alon...,nytimes.com,link,shared_story,428,156,...,http://www.nytimes.com/2012/09/13/world/middle...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-12 16:10:11,https://www.nytimes.com/2012/09/13/world/middl...,1.0,1.0,NaN,NaN,1.0,1.0
67,5281959998_416313445091780,5281959998,Fed Announces New Round of Bond Buying to Spur...,The Federal Reserve opened a new chapter Thurs...,The Federal Reserve said it would expand its h...,nytimes.com,link,shared_story,334,114,...,http://www.nytimes.com/2012/09/14/business/eco...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-13 17:23:21,https://www.nytimes.com/2012/09/14/business/ec...,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47840,5281959998_10150957971334999,5281959998,The New York Times - Theater,"We're with Mike Daisey, who is performing a po...","Learn about the making of Donald J. Trump, as ...",NaN,video,shared_story,229,59,...,https://www.facebook.com/nytimestheater/videos...,https://scontent.xx.fbcdn.net/v/t15.0-10/s130x...,2016-11-07 16:31:11,https://www.facebook.com/nytimestheater/videos...,NaN,NaN,NaN,NaN,1.0,1.0
47845,5281959998_10150958061204999,5281959998,Nicholas Kristof,"Nicholas Kristof, a columnist for The New York...",Our last Facebook Live before the election! Ch...,NaN,video,shared_story,144,40,...,https://www.facebook.com/kristof/videos/101545...,https://scontent.xx.fbcdn.net/v/t15.0-10/s130x...,2016-11-07 17:45:15,https://www.facebook.com/kristof/videos/101545...,NaN,NaN,NaN,NaN,1.0,1.0
47847,5281959998_10150958085764999,5281959998,The New York Times - Well - Health,Lauren Hard and Paul Moon of The New York Time...,"We're at Pure Yoga with Sarah Attia, founder o...",NaN,video,shared_story,70,6,...,https://www.facebook.com/WellNYT/videos/116561...,https://scontent.xx.fbcdn.net/v/t15.0-10/s130x...,2016-11-07 18:06:08,https://www.facebook.com/WellNYT/videos/116561...,NaN,NaN,NaN,NaN,1.0,1.0
47850,5281959998_10150958116574999,5281959998,Two Men Entered the Ring for Their First Profe...,A reason to binge on some of our best long rea...,Anthony Taylor had won the fight. And the fall...,nytimes.com,link,shared_story,87,10,...,http://www.nytimes.com/2016/03/28/sports/boxin...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-07 19:10:00,htt

I can now try to join the facebook posts to articles using link, description, and name, and I can isolate only the posts that have no duplicates in each joining column.

In [25]:
# separate out main headline from headline dict column
df_nyt['main_headline'] = df_nyt['headline'].map(lambda x: x['main'])

### Test join on link

In [26]:
len(df.loc[df['dupes_on_link'].isna()])

41664

In [27]:
# Test joining together the matches based on trimmed link, starting with only
# facebook posts that are not duplicated based on link
df_link_matches = pd.merge(left=df.loc[df['dupes_on_link'].isna()], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='trim_link', 
                           right_on='web_url',
                           suffixes=('_nyt', '_nyt'))
df_link_matches.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
0,5281959998_416259568436280,5281959998,Did Barack Obama Save Ohio?,"'There are several pivotal states — Ohio, Flor...",Why the battle to take credit for Ohio’s ever-...,nytimes.com,link,shared_story,413,44,...,2012-09-05T09:00:27+0000,article,Magazine,Magazine,"{'original': 'By Matt Bai', 'person': [{'first...",News,nyt://article/b122515f-2281-5ca8-8749-7a64f317...,7113,nyt://article/b122515f-2281-5ca8-8749-7a64f317...,Did Barack Obama Save Ohio?
1,5281959998_268320586619940,5281959998,The Weatherman Is Not a Moron,"In 1940, the chance of an American being kille...",In the hocus-pocus realm of predicting the fut...,nytimes.com,link,shared_story,338,14,...,2012-09-07T09:00:29+0000,article,Magazine,Magazine,"{'original': 'By Nate Silver', 'person': [{'fi...",News,nyt://article/c45ee260-62ac-52cf-8937-29d1f731...,3915,nyt://article/c45ee260-62ac-52cf-8937-29d1f731...,The Weatherman Is Not a Moron
2,5281959998_463550620334374,5281959998,Pot for Parents,'When I’m rolling around the floor with my gig...,"I am a more loving, attentive and patient fath...",nytimes.com,link,shared_story,502,83,...,2012-09-07T23:53:19+0000,article,OpEd,Opinion,"{'original': 'By Mark Wolfe', 'person': [{'fir...",Op-Ed,nyt://article/18ecc9e3-6e52-5c51-a7c2-9a788727...,855,nyt://article/18ecc9e3-6e52-5c51-a7c2-9a788727...,Pot for Parents
3,5281959998_437214569655310,5281959998,Debt Collectors Cashing In on Student Loan Rou...,Quotation of the Day: 'You are going to pay it...,The Department of Education last year paid $1....,nytimes.com,link,shared_story,260,74,...,2012-09-08T22:17:12+0000,article,Business,Business Day,"{'original': 'By Andrew Martin', 'person': [{'...",News,nyt://article/28e0313f-8660-56b8-9ed8-db9be898...,2508,nyt://article/28e0313f-8660-56b8-9ed8-db9be898...,Debt Collectors Cashing In on Student Loans
4,5281959998_481450601873461,5281959998,New Rules,"'The truth is, if you want a decent job that w...",The plan to “work hard and play by the rules” ...,nytimes.com,link,shared_story,3073,260,...,2012-09-08T17:35:05+0000,article,Editorial,Opinion,"{'original': 'By Thomas L. Friedman', 'person'...",Op-Ed,nyt://article/a6deff8e-29dc-5e71-b52a-20a81006...,915,nyt://article/a6deff8e-29dc-5e71-b52a-20a81006...,New Rules


In [28]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_link_matches[df_link_matches.duplicated(subset=['id'])]))
df_link_matches[df_link_matches.duplicated(subset=['id'], keep=False)]


1


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
3622,5281959998_10150608613634999,5281959998,Marc Maron Discusses His Podcast Interview Wit...,"Marc Maron, the idiosyncratic, inquisitive an...",The comedian interviewed the commander in chie...,artsbeat.blogs.nytimes.com,link,shared_story,415,24,...,2015-06-19T20:10:35+0000,article,Culture,Arts,"{'original': 'By Dave Itzkoff', 'person': [{'f...",News,nyt://article/807c0c24-e084-54cf-8564-27b2d2c8...,1667,nyt://article/807c0c24-e084-54cf-8564-27b2d2c8...,Marc Maron Discusses His Podcast Interview Wit...
3623,5281959998_10150608613634999,5281959998,Marc Maron Discusses His Podcast Interview Wit...,"Marc Maron, the idiosyncratic, inquisitive an...",The comedian interviewed the commander in chie...,artsbeat.blogs.nytimes.com,link,shared_story,415,24,...,2015-06-19T20:10:35+0000,article,,Crosswords & Games,"{'original': 'By DAVE ITZKOFF', 'person': [{'f...",News,nyt://article/8d1285ea-4f3a-57d0-8c0b-8923ac69...,1667,nyt://article/8d1285ea-4f3a-57d0-8c0b-8923ac69...,Marc Maron Discusses His Podcast Interview Wit...


Cool, just one duplicate, looks like the NYT version had this listed twice.

In [29]:
# drop duplicates
df_link_matches.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [31]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_link_matches[df_link_matches.duplicated(subset=['_id'])]))
df_link_matches[df_link_matches.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline


In [32]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_link_matches['id'].values), 'matched_on'] = 'link'
len(df[df['matched_on']=='link'])

5819

### Test join on description

In [33]:
# pull only posts without duplicate descriptions, and where the posts
# haven't already been matched with an article
len(df.loc[(df['dupes_on_desc'].isna()) & (df['matched_on'].isna())])

33778

In [34]:
# Test joining together the matches based on description and snippet
df_desc_matches = pd.merge(left=df.loc[(df['dupes_on_desc'].isna()) 
                                       & (df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='description', 
                           right_on='snippet',
                           suffixes=('_nyt', '_nyt'))
len(df_desc_matches)

24344

In [35]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_desc_matches[df_desc_matches.duplicated(subset=['id'])]))
df_desc_matches[df_desc_matches.duplicated(subset=['id'], keep=False)]


586


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
176,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,2013-05-18T02:48:10+0000,multimedia,,Fashion & Style,"{'original': 'Joanna Nikas', 'person': [{'firs...",Video,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087,0,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087,Bill Cunningham | Full Bloom
177,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,2013-05-20T16:54:55+0000,article,,Style,"{'original': 'By Bill Cunningham', 'person': [...",News,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...,47,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...,"Video: Bill Cunningham’s On the Street, Full B..."
206,5281959998_10150309977124999,5281959998,Video: Escape From North Korea,'The government doesn't give us a thing but th...,A smuggler named Dragon attempts to guide Nort...,nytimes.com,link,shared_story,386,20,...,2013-06-06T01:05:08+0000,multimedia,Opinion,Opinion,"{'original': 'Ann Shin', 'person': [{'firstnam...",Video,nyt://video/786833f5-cd66-5444-9cdf-75707417146a,0,nyt://video/786833f5-cd66-5444-9cdf-75707417146a,Escape From North Korea
207,5281959998_10150309977124999,5281959998,Video: Escape From North Korea,'The government doesn't give us a thing but th...,A smuggler named Dragon attempts to guide Nort...,nytimes.com,link,shared_story,386,20,...,2013-06-06T01:12:22+0000,article,OpEd,Opinion,"{'original': 'By Ann Shin', 'person': [{'first...",Op-Ed,nyt://article/779326d9-6752-5ff6-95a3-31ef22ba...,201,nyt://article/779326d9-6752-5ff6-95a3-31ef22ba...,‘Escape From North Korea’
290,5281959998_10150324158779999,5281959998,Video: Tunnel Vision,Was your commute this morning this transporting?,A short impressionistic film shot from New Yor...,nytimes.com,link,shared_story,186,12,...,2013-07-30T01:10:14+0000,multimedia,Opinion,Opinion,"{'original': 'Jeff Scher', 'person': [{'firstn...",Video,nyt://video/2222d988-9f30-5621-99fe-11d5c7df296b,0,nyt://video/2222d988-9f30-5621-99fe-11d5c7df296b,Tunnel Vision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23875,5281959998_10150903747994999,5281959998,9 New Books We Recommend This Week,Time to dig into the meaty books of fall.,Suggested reading by the editors of The New Yo...,nytimes.com,link,shared_story,616,31,...,2016-09-29T18:44:09+0000,article,BookReview,Books,"{'original': '', 'person': [], 'organization':...",News,nyt://article/5f01ebd6-67f0-5fc9-bc7e-bd1ba449...,710,nyt://article/5f01ebd6-67f0-5fc9-bc7e-bd1ba449...,9 New Books We Recommend This Week
23927,5281959998_10150907252139999,5281959998,The Many Sad Fates of Mr. Toledano,Phillip Toledano was terrified of growing old....,Phillip Toledano was terrified of growing old....,nytimes.com,link,shared_story,1449,122,...,2016-09-20T20:08:21+0000,article,OpEd,Opinion,"{'original': 'By Joshua Seftel', 'person': [{'...",Op-Ed,nyt://article/6ea8cbcd-bcb3-5d8c-b178-641c6ef2...,492,nyt://article/6ea8cbcd-bcb3-5d8c-b178-641c6ef2...,The Many Sad Fates of Mr. Toledano
23928,5281959998_10150907252139999,5281959998,The Many Sad Fates of Mr. Toledano,Phillip Toledano was terrified of growing old....,Phillip Toledano was terrified of growing old....,nytimes.com,link,shared_story,1449,122,...,2016-09-20T20:09:42+0000,multimedia,Opinion,Opinion,"{'original': 'By Joshua Seftel', 'person': [{'...",Video,nyt://video/e3573032-dee9-5fb6-92bf-d84e0ab83b9a,0,nyt://video/e3573032-dee9-5fb6-92bf-d84e0ab83b9a,The Many Sad Fates of Mr. Toledano
24007,5281959998_10150909556049999,5281959998,

Definitely more duplicates here. It looks like they're primarily due to there being a video version and a news/written version of certain pieces.

Looking at a few examples, can I easily tell whether the facebook post is for one or the other? I may need to write my own custom deduplication script to identify the best one to keep, especially if only certain ones will have comments I can use to compare to facebook likes and shares.

In [36]:
df_desc_matches.loc[[23927, 23928]].values

array([['5281959998_10150907252139999', 5281959998,
        'The Many Sad Fates of Mr. Toledano',
        'Phillip Toledano was terrified of growing old. So he decided to do it as many times as possible. (via The New York Times Opinion Section)',
        'Phillip Toledano was terrified of growing old. So he decided to do it as many times as possible.',
        'nytimes.com', 'link', 'shared_story', 1449, 122, 507, 116, 86,
        3, 7, 0, 1, 'http://nyti.ms/2d9wfwS',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQDbRDPCaFYKQd-v&w=130&h=130&url=https%3A%2F%2Fstatic01.nyt.com%2Fimages%2F2016%2F09%2F12%2Fmultimedia%2Fopdoc-toledano%2Fopdoc-toledano-facebookJumbo.jpg&cfs=1&sx=258&sy=0&sw=549&sh=549',
        '2016-09-21 01:55:00', 'https://nyti.ms/2d9wfwS', nan, nan, nan,
        nan, nan, nan, nan,
        'Phillip Toledano was terrified of growing old. So he decided to do it as many times as possible.',
        'https://www.nytimes.com/2016/09/20/opinion/the-many-sad-fates-of

So the video link here https://www.nytimes.com/video/opinion/100000004653100/the-many-sad-fates-of-mr-toledano.html is the video only, and it doesn't include any comments listed on the page.

But the article link to the text description does include comments https://www.nytimes.com/2016/09/20/opinion/the-many-sad-fates-of-mr-toledano.html.

What I may do is just keep both copies in for now, since I can't know for sure which, if either, will have comments I can pull via APIs or scrape from the website. I'll perform that step knowing there will be some duplicates, and then keep whichever version got comments, if one did.

In [37]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_desc_matches[df_desc_matches.duplicated(subset=['_id'])]))
df_desc_matches[df_desc_matches.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline


In [38]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_desc_matches['id'].values), 'matched_on'] = 'desc'
len(df[df['matched_on']=='desc'])

23758

In [39]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

13782

In [40]:
# how many unmatched are not duplicates on something?
len(df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)])


8764

In [41]:
df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,picture,posted_at,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:44:02,https://www.nytimes.com/2012/09/07/opinion/rog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,5281959998_224679674326401,5281959998,Video: Patriot Game,'POWER YOUR PRESIDENT TO WINNING. USE MANY GOO...,The Gregory Brothers present a video game-insp...,nytimes.com,link,shared_story,178,38,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-17 16:15:10,https://www.nytimes.com/video/2012/09/16/opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,5281959998_209496975846889,5281959998,The New World,"By the beginning of the 21st century, the worl...",An interactive series of maps show possible ne...,nytimes.com,link,shared_story,228,32,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-24 13:31:40,https://www.nytimes.com/interactive/2012/09/23...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,5281959998_106658229491567,5281959998,Foxconn Plant in China Closed After Worker Riot,"Foxconn Technology, a major supplier to some o...",A spokesman said a fight among employees on Su...,nytimes.com,link,shared_story,124,22,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-24 16:44:29,https://www.nytimes.com/2012/09/25/technology/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,5281959998_447295091981043,5281959998,N.F.L. and Referees Are Close to a Deal,The N.F.L. and the referees’ union are close t...,The N.F.L. and the referees’ union are close t...,nytimes.com,link,shared_story,172,14,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-26 16:41:57,https://www.nytimes.com/2012/09/27/sports/foot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47770,5281959998_10150955899794999,5281959998,NYT Watching Recommendation of the Day: 'Child...,'Children of Men' feels prescient and frighten...,Watch if you like human-scale dystopian scienc...,nytimes.com,link,shared_story,2716,204,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 04:10:00,https://www.nytimes.com/watching/recommendatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47778,5281959998_10150953832879999,5281959998,Thanksgiving Dinner: Ideas and Tips,How to pull off a perfect Thanksgiving feast.,There is no more inclusive holiday on the Amer...,cooking.nytimes.com,link,shared_story,371,36,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 13:55:00,https://cooking.nytimes.com/thanksgiving/dinne...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47844,5281959998_10150957984614999,5281959998,Instagram photo by New York Times Archives • N...,"1917: “Women of all ages, from the nearly feeb...",See this Instagram photo by @nytarchives • 63 ...,instagram.com,link,shared_story,23320,257,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-07 17:40:01,https://www.instagram.com/p/BMgr8zwBZwQ/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-07 18:25:01,https://cooking.nytimes.com/recipes/1017780-wh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.loc[[2]].values

array([['5281959998_467433813289503', 5281959998, 'The Organic Fable',
        "'Organic has long since become an ideology, the romantic back-to-nature obsession of an upper middle class able to afford it and oblivious, in their affluent narcissism, to the challenge of feeding a planet whose population will surge to 9 billion,' writes Op-Ed columnist Roger Cohen.",
        'A study exposes the hype behind a pseudoscientific ideology. The future is nonorganic.',
        'nytimes.com', 'link', 'shared_story', 812, 290, 421, 0, 0, 0, 0,
        0, 0,
        'http://www.nytimes.com/2012/09/07/opinion/roger-cohen-the-organic-fable.html',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQA6AciF5lLAnN7B&w=130&h=130&url=http%3A%2F%2Fgraphics8.nytimes.com%2Fimages%2F2012%2F09%2F06%2Fopinion%2Fiht-edcohen07-illo%2Fiht-edcohen07-illo-thumbLarge.jpg&cfs=1',
        '2012-09-08 16:44:02',
        'https://www.nytimes.com/2012/09/07/opinion/roger-cohen-the-organic-fable.html',
        nan, 

In [43]:
df_nyt.loc[(df_nyt['headline'].str.contains('Fable', regex=False))]



,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline


### Test join on headline

In [44]:
# pull only posts without duplicate names, and where the posts
# haven't already been matched with an article
len(df.loc[(df['dupes_on_name'].isna()) & (df['matched_on'].isna())])

10423

In [45]:
# Test joining together the matches based on description and snippet
df_name_matches = pd.merge(left=df.loc[(df['dupes_on_name'].isna()) 
                                       & (df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='name', 
                           right_on='main_headline',
                           suffixes=('_nyt', '_nyt'))
len(df_name_matches)

3488

In [46]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_name_matches[df_name_matches.duplicated(subset=['id'])]))
df_name_matches[df_name_matches.duplicated(subset=['id'], keep=False)]


232


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
1,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2012-09-30T23:21:31+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/a923a469-ffbb-56a1-bfa8-d69b10...,0,nyt://slideshow/a923a469-ffbb-56a1-bfa8-d69b10...,Global Citizen Festival
2,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2013-09-29T21:57:57+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/2f1ad1cc-da0f-596b-b7fc-90e594...,0,nyt://slideshow/2f1ad1cc-da0f-596b-b7fc-90e594...,Global Citizen Festival
3,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2015-09-27T21:11:14+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/abc08ebc-c02a-5aac-8780-d4698c...,0,nyt://slideshow/abc08ebc-c02a-5aac-8780-d4698c...,Global Citizen Festival
25,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By THE NEW YORK TIMES', 'person'...",News,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...,1915,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...,Q. and A. on the Papal Transition
26,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By The New York Times', 'person'...",News,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...,1915,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...,Q. and A. on the Papal Transition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,5281959998_10150856491074999,5281959998,The Trans-Pacific Partnership Trade Accord Exp...,Need a rundown on the Trans-Pacific Partnershi...,"The giant trade pact, years in the making, has...",nytimes.com,link,shared_story,256,52,...,2016-07-26T22:36:25+0000,article,Business,Business Day,"{'original': 'By Kevin Granville', 'person': [...",News,nyt://article/7f176cc0-5c9f-5d11-98a1-1adfdbe3...,1326,nyt://article/7f176cc0-5c9f-5d11-98a1-1adfdbe3...,The Trans-Pacific Partnership Trade Accord Exp...
3185,5281959998_10150859876799999,5281959998,"The Week in Pictures: July 29, 2016","A look back at the week, in photos.",Photos from The New York Times and photographe...,nytimes.com,link,shared_story,79,7,...,2016-07-29T06:06:22+0000,multimedia,Blogs,Blogs,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/3fc2fd1a-f802-5a0f-b350-571bca...,0,nyt://slideshow/3fc2fd1a-f802-5a0f-b350-571bca...,"The Week in Pictures: July 29, 2016"
3186,5281959998_10150859876799999,5281959998,"The Week in Pictures: July 29, 2016","A look back at the week, in photos.",Photos from The New York Times and photographe...,nytimes.com,link,shared_story,79,7,...,2016-07-29T07:00:24+0000,article,,Multimedia/Photos,"{'original': 'By The New York Times', 'person'...",News,nyt://article/bd05466a-7929-509b-8ef3-a31a822a...,25,nyt://article/bd05466a-7929-509b-8ef3-a31a822a...,"The Week in Pictures: July 29, 2016"
3426,5281959998_1015090

These look like duplicates for the same reasons the others were; multiple copies of a feature as different types of media, or perhaps updated like with slideshows where there may have been multiple versions.

In [47]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_name_matches[df_name_matches.duplicated(subset=['_id'])]))
df_name_matches[df_name_matches.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline


In [48]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_name_matches['id'].values), 'matched_on'] = 'name'
len(df[df['matched_on']=='name'])

3256

In [49]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

10526

In [50]:
# unmatched and not duplicates of any sort I've tested
df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,picture,posted_at,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:44:02,https://www.nytimes.com/2012/09/07/opinion/rog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,5281959998_224679674326401,5281959998,Video: Patriot Game,'POWER YOUR PRESIDENT TO WINNING. USE MANY GOO...,The Gregory Brothers present a video game-insp...,nytimes.com,link,shared_story,178,38,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-17 16:15:10,https://www.nytimes.com/video/2012/09/16/opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,5281959998_106658229491567,5281959998,Foxconn Plant in China Closed After Worker Riot,"Foxconn Technology, a major supplier to some o...",A spokesman said a fight among employees on Su...,nytimes.com,link,shared_story,124,22,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-24 16:44:29,https://www.nytimes.com/2012/09/25/technology/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,5281959998_447295091981043,5281959998,N.F.L. and Referees Are Close to a Deal,The N.F.L. and the referees’ union are close t...,The N.F.L. and the referees’ union are close t...,nytimes.com,link,shared_story,172,14,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-26 16:41:57,https://www.nytimes.com/2012/09/27/sports/foot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,5281959998_151621574982436,5281959998,Live Updates From the Presidential Debate in D...,"Follow the debate with live video, updates and...",Updates and analysis from New York Times repor...,elections.nytimes.com,link,shared_story,198,21,...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-04 01:11:27,https://elections.nytimes.com/2012/debates/pre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47770,5281959998_10150955899794999,5281959998,NYT Watching Recommendation of the Day: 'Child...,'Children of Men' feels prescient and frighten...,Watch if you like human-scale dystopian scienc...,nytimes.com,link,shared_story,2716,204,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 04:10:00,https://www.nytimes.com/watching/recommendatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47778,5281959998_10150953832879999,5281959998,Thanksgiving Dinner: Ideas and Tips,How to pull off a perfect Thanksgiving feast.,There is no more inclusive holiday on the Amer...,cooking.nytimes.com,link,shared_story,371,36,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 13:55:00,https://cooking.nytimes.com/thanksgiving/dinne...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47844,5281959998_10150957984614999,5281959998,Instagram photo by New York Times Archives • N...,"1917: “Women of all ages, from the nearly feeb...",See this Instagram photo by @nytarchives • 63 ...,instagram.com,link,shared_story,23320,257,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-07 17:40:01,https://www.instagram.com/p/BMgr8zwBZwQ/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-07 18:25:01,https://cooking.nytimes.com/recipes/1017780-wh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As a final step, I'm going to try to pull the year-month-day out of the links in both the Facebook and NYT API links, and match where both the description/name and the date from the link matches.

The publication date and date when an article was posted to Facebook sometimes varies, since the articles are sometimes posted to Facebook the next day or later, so I think using the date in the link should be the best match.

### Pull date out of link for Facebook posts

In [51]:
#df.drop(columns=['link_date'], inplace=True)

In [52]:
df = pattern_match_in_df(df, doc_col='trim_link', hit_col='link_date', 
                pattern=r"/(201[2-6]/[01][0-9]/[0-3][0-9])/", 
                out_type='string', replace=False)

In [53]:
df[['trim_link', 'link_date']][450:500]

,trim_link,link_date
495,https://opinionator.blogs.nytimes.com/2012/10/...,2012/10/24
497,https://www.nytimes.com/2012/10/26/nyregion/gi...,2012/10/26
498,https://thelede.blogs.nytimes.com/2012/10/25/a...,2012/10/25
499,https://well.blogs.nytimes.com/2012/10/24/laug...,2012/10/24
500,https://www.nytimes.com/2012/10/26/world/asia/...,2012/10/26
501,https://www.nytimes.com/2012/10/26/us/politics...,2012/10/26
502,https://www.nytimes.com/2012/10/26/business/gl...,2012/10/26
503,https://cn.nytimes.com/article/china/2012/10/2...,2012/10/26
504,https://www.nytimes.com/2012/10/27/business/ec...,2012/10/27
505,https://www.nytimes.com/2012/10/26/business/in...,2012/10/26


### Pull date out of link for NYT articles

In [54]:
df_nyt.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
0,The best New Year’s Day news for Chicago’s spo...,https://www.nytimes.com/2012/01/01/sports/look...,The best New Year’s Day news for Chicago’s spo...,Dan McGrath writes a column for The Chicago Ne...,A,25B,The New York Times,"[{'rank': 0, 'subtype': 'jumbo', 'caption': No...",{'main': 'A Year Full of Missteps Is Finally O...,"[{'name': 'organizations', 'value': 'Chicago C...",2012-01-01T00:00:28+0000,article,National,Sports,"{'original': 'By Dan McGrath', 'person': [{'fi...",News,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...,912,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...,A Year Full of Missteps Is Finally Over
1,The man was preparing to board a flight in Mid...,https://www.nytimes.com/2012/01/01/us/servicem...,The man was preparing to board a flight in Mid...,HOUSTON — A man preparing to board a flight at...,A,19,The New York Times,[],{'main': 'Serviceman Held for Carrying Explosi...,"[{'name': 'subject', 'value': 'Airport Securit...",2012-01-01T00:09:12+0000,article,National,U.S.,"{'original': 'By Manny Fernandez', 'person': [...",News,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...,242,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...,Serviceman Held for Carrying Explosives Into T...
2,"Setting the scene, Hollywood style, for what c...",https://fifthdown.blogs.nytimes.com/2011/12/31...,"Setting the scene, Hollywood style, for what c...","Yes, the Jets are a long shot to qualify for t...",NaN,NaN,The New York Times,[],{'main': 'Picturing How Ryan Brothers Could Se...,"[{'name': 'subject', 'value': 'Football', 'ran...",2012-01-01T00:14:59+0000,article,,Sports,"{'original': 'By George Bretherton', 'person':...",News,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...,491,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...,Picturing How Ryan Brothers Could Settle Fami...
3,Experts said the broadcasts were most likely p...,https://thecaucus.blogs.nytimes.com/2011/12/31...,Experts said the broadcasts were most likely p...,DES MOINES — Can they do that?,NaN,NaN,The New York Times,[],{'main': 'Experts Weigh In on Outside Groups’ ...,"[{'name': 'subject', 'value': 'Campaign Financ...",2012-01-01T00:15:52+0000,article,,U.S.,"{'original': 'By Nicholas Confessore', 'person...",News,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...,449,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...,Experts Weigh In on Outside Groups’ Pro-Gingri...
4,After more than a decade in which San Francisc...,https://www.nytimes.com/2012/01/01/us/an-unlik...,After more than a decade in which San Francisc...,"Susan Beckstead stepped out of her sky-blue, t...",A,25A,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'An Unlikely Group Rebels Against Pre...,"[{'name': 'subject', 'value': 'Historic Buildi...",2012-01-01T00:15:54+0000,article,National,U.S.,"{'original': 'By Matt Smith', 'person': [{'fir...",News,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...,1164,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...,An Unlikely Group Rebels Against Preservation ...


In [55]:
df_nyt = pattern_match_in_df(df_nyt, doc_col='web_url', hit_col='link_date', 
                pattern=r"/(201[2-6]/[01][0-9]/[0-3][0-9])/", 
                out_type='string', replace=False)

### Join on dates and other fields

In [57]:
df_name_matches2 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                        (df['link_date'].isna()==False) &
                                        (df['link_date'] != '')], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['name', 'link_date'], 
                           right_on=['main_headline', 'link_date'],
                           suffixes=('_nyt', '_nyt'))
df_name_matches2.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
0,5281959998_480453791987573,5281959998,"Unfriending Someone, Before Facebook","When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...",nytimes.com,link,shared_story,454,41,...,2012-10-03T09:00:09+0000,article,Styles,Booming,"{'original': 'By Joyce Wadler', 'person': [{'f...",News,nyt://article/820edbd0-93ac-59f2-9567-6556e30a...,971,nyt://article/820edbd0-93ac-59f2-9567-6556e30a...,"Unfriending Someone, Before Facebook"
1,5281959998_429781297079467,5281959998,"Unfriending Someone, Before Facebook",Is unfriending a person on Facebook the right ...,"How much better things were 50 years ago, when...",nytimes.com,link,shared_story,461,69,...,2012-10-03T09:00:09+0000,article,Styles,Booming,"{'original': 'By Joyce Wadler', 'person': [{'f...",News,nyt://article/820edbd0-93ac-59f2-9567-6556e30a...,971,nyt://article/820edbd0-93ac-59f2-9567-6556e30a...,"Unfriending Someone, Before Facebook"
2,5281959998_437277122974113,5281959998,512 Paths to the White House,Select President Barack Obama or Mitt Romney i...,Explore the routes through the electoral battl...,nytimes.com,link,shared_story,3605,578,...,2012-11-03T01:06:20+0000,multimedia,U.S.,U.S.,"{'original': '', 'person': [], 'organization':...",Interactive Feature,nyt://interactive/c239a213-7793-5311-88bb-2e6a...,0,nyt://interactive/c239a213-7793-5311-88bb-2e6a...,512 Paths to the White House
3,5281959998_448173665241208,5281959998,512 Paths to the White House,Obama and Romney have 512 paths to the White H...,Explore the routes through the electoral battl...,nytimes.com,link,shared_story,3783,506,...,2012-11-03T01:06:20+0000,multimedia,U.S.,U.S.,"{'original': '', 'person': [], 'organization':...",Interactive Feature,nyt://interactive/c239a213-7793-5311-88bb-2e6a...,0,nyt://interactive/c239a213-7793-5311-88bb-2e6a...,512 Paths to the White House
4,5281959998_229429513854758,5281959998,Clashes Break Out After Morsi Seizes New Power...,Opponents of President Mohamed Morsi were repo...,Protesters were said to have set fire to the o...,nytimes.com,link,shared_story,194,22,...,2012-11-23T14:34:15+0000,article,Foreign,World,{'original': 'By Kareem Fahim and David D. Kir...,News,nyt://article/ebff0851-b7bc-586c-9f2f-1cf3c9ab...,1244,nyt://article/ebff0851-b7bc-586c-9f2f-1cf3c9ab...,Clashes Break Out After Morsi Seizes New Power...


In [58]:
df_name_matches2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 0 to 66
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                67 non-null     object 
 1   page_id           67 non-null     int64  
 2   name              67 non-null     object 
 3   message           67 non-null     object 
 4   description       67 non-null     object 
 5   caption           67 non-null     object 
 6   post_type         67 non-null     object 
 7   status_type       67 non-null     object 
 8   likes_count       67 non-null     int64  
 9   comments_count    67 non-null     int64  
 10  shares_count      67 non-null     int64  
 11  love_count        67 non-null     int64  
 12  wow_count         67 non-null     int64  
 13  haha_count        67 non-null     int64  
 14  sad_count         67 non-null     int64  
 15  thankful_count    67 non-null     int64  
 16  angry_count       67 non-null     int64  
 17 

In [59]:
df_name_matches2[['message', 'description', 'name', 'snippet', 'likes_count',
                  'comments_count', 'shares_count',
                  'main_headline', 'link_date', 'posted_at']]

,message,description,name,snippet,likes_count,comments_count,shares_count,main_headline,link_date,posted_at
0,"When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",454,41,195,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-03 16:10:40
1,Is unfriending a person on Facebook the right ...,"How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",461,69,196,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-04 20:17:42
2,Select President Barack Obama or Mitt Romney i...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3605,578,1435,512 Paths to the White House,2012/11/02,2012-11-03 17:36:30
3,Obama and Romney have 512 paths to the White H...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3783,506,1177,512 Paths to the White House,2012/11/02,2012-11-06 16:49:58
4,Opponents of President Mohamed Morsi were repo...,Protesters were said to have set fire to the o...,Clashes Break Out After Morsi Seizes New Power...,Thousands of protesters chanted for the downfa...,194,22,40,Clashes Break Out After Morsi Seizes New Power...,2012/11/24,2012-11-23 16:11:39
...,...,...,...,...,...,...,...,...,...,...
62,"' 'Moonlight' dwells on the dignity, beauty an...",Our critic A.O. Scott says this film about a y...,‘Moonlight’: Is This the Year’s Best Movie?,Our critic A.O. Scott says this film about a y...,454,24,105,‘Moonlight’: Is This the Year’s Best Movie?,2016/10/21,2016-10-24 01:55:00
63,"'Donald J. Trump has done one good thing,' Joh...",John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,403,51,53,Making Latino Life Visible,2016/10/21,2016-10-21 19:25:00
64,'Latinos need to demand our place in American ...,John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,477,46,102,Making Latino Life Visible,2016/10/21,2016-10-24 02:15:00
65,Estonia is using a form of 'military sport' to...,'The guerrilla activity should start on occupi...,"Spooked by Russia, Tiny Estonia Trains a Natio...","A NATO member, Estonia is using an improbably ...",1551,124,675,"Spooked by Russia, Tiny Estonia Trains a Natio...",2016/11/01,2016-11-01 07:25:00


In [60]:
df_desc_matches2 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                        (df['link_date'].isna()==False) &
                                        (df['link_date'] != '')], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['description', 'link_date'], 
                           right_on=['snippet', 'link_date'],
                           suffixes=('_nyt', '_nyt'))
len(df_desc_matches2)

62

In [61]:
df_desc_matches2[['message', 'description', 'name', 'snippet', 'likes_count',
                  'comments_count', 'shares_count',
                  'main_headline', 'link_date', 'posted_at']]

,message,description,name,snippet,likes_count,comments_count,shares_count,main_headline,link_date,posted_at
0,"When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",454,41,195,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-03 16:10:40
1,Is unfriending a person on Facebook the right ...,"How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",461,69,196,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-04 20:17:42
2,The U.S. economy grew at an annual rate of 2 p...,More positive consumer activity and a healthie...,U.S. Economy Grew at 2% Rate in 3rd Quarter,More positive consumer activity and a healthie...,730,72,165,U.S. Growth Rate Picks Up to 2%,2012/10/27,2012-10-26 12:53:54
3,Select President Barack Obama or Mitt Romney i...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3605,578,1435,512 Paths to the White House,2012/11/02,2012-11-03 17:36:30
4,Obama and Romney have 512 paths to the White H...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3783,506,1177,512 Paths to the White House,2012/11/02,2012-11-06 16:49:58
...,...,...,...,...,...,...,...,...,...,...
57,'Moonlight' is about as beautiful a movie as y...,Our critic A.O. Scott says this film about a y...,‘Moonlight’: Is This the Year’s Best Movie?,Our critic A.O. Scott says this film about a y...,1734,98,477,‘Moonlight’: Is This the Year’s Best Movie?,2016/10/21,2016-10-21 02:25:35
58,"' 'Moonlight' dwells on the dignity, beauty an...",Our critic A.O. Scott says this film about a y...,‘Moonlight’: Is This the Year’s Best Movie?,Our critic A.O. Scott says this film about a y...,454,24,105,‘Moonlight’: Is This the Year’s Best Movie?,2016/10/21,2016-10-24 01:55:00
59,"'Donald J. Trump has done one good thing,' Joh...",John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,403,51,53,Making Latino Life Visible,2016/10/21,2016-10-21 19:25:00
60,'Latinos need to demand our place in American ...,John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,477,46,102,Making Latino Life Visible,2016/10/21,2016-10-24 02:15:00


In [62]:
# update matched status for these two
# Link wasn't used since we're adding the extra factor of matching on date 
# from the link, so it shouldn't be different
df.loc[df['id'].isin(df_name_matches2['id'].values), 'matched_on'] = 'name'

df.loc[df['id'].isin(df_desc_matches2['id'].values), 'matched_on'] = 'desc'

In [63]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

10448

## Initial Matches Assembly


In [64]:
df_list = [df_link_matches, df_desc_matches, df_name_matches,
           df_desc_matches2, df_name_matches2]

df_matches = pd.concat(df_list, axis=0, join='inner', ignore_index=True, )

df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33780 entries, 0 to 33779
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                33780 non-null  object 
 1   page_id           33780 non-null  int64  
 2   name              33780 non-null  object 
 3   message           32682 non-null  object 
 4   description       33655 non-null  object 
 5   caption           33723 non-null  object 
 6   post_type         33780 non-null  object 
 7   status_type       33780 non-null  object 
 8   likes_count       33780 non-null  int64  
 9   comments_count    33780 non-null  int64  
 10  shares_count      33780 non-null  int64  
 11  love_count        33780 non-null  int64  
 12  wow_count         33780 non-null  int64  
 13  haha_count        33780 non-null  int64  
 14  sad_count         33780 non-null  int64  
 15  thankful_count    33780 non-null  int64  
 16  angry_count       33780 non-null  int64 

In [65]:
match_links1 = df_matches['web_url'].unique()
len(match_links1)

33627

In [88]:
# export match links 1
pd.DataFrame(match_links1).to_csv("match_links1.csv")

In [5]:
# later, load both versions so we can compare
match_links1 = pd.read_csv('data/match_links1.csv', index_col=0, names=['link'], 
                           skiprows=1)

match_links2 = pd.read_csv('data/match_links2.csv', index_col=0, names=['link'], 
                           skiprows=1)

In [6]:
print(len(match_links1))
print(len(match_links2))

33627
36551


In [7]:
# get links in match 2 not in match 1
match_links2_diff = set(match_links2['link']) - set(match_links1['link'])
len(match_links2_diff)

3144

In [8]:
match_links2_diff = list(match_links2_diff)
match_links2_diff[:5]

['https://www.nytimes.com/2013/12/26/health/common-knee-surgery-does-very-little-for-some-study-suggests.html',
 'https://www.nytimes.com/slideshow/2016/07/23/world/deadly-shooting-at-munich-mall.html',
 'https://thelede.blogs.nytimes.com/2014/01/22/footage-of-mayhem-on-the-streets-of-kiev/',
 'https://www.nytimes.com/interactive/2014/02/02/fashion/red-carpet-project.html',
 'https://www.nytimes.com/2016/09/15/briefing/hillary-clinton-donald-trump-edward-snowden.html']

# Scrape Comment Counts

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib
#%pip install html5lib
#%pip install lxml

In [2]:
driver = webdriver.Chrome('/Users/jessicamiles/Downloads/chromedriver')

In [3]:
driver.get('https://www.nytimes.com/2016/10/31/opinion/strangers-on-a-strange-election.html')

In [64]:
print(driver.current_url)

http://archive.nytimes.com/www.nytimes.com/interactive/2012/10/28/nyregion/hurricane-sandy.html


In [77]:
com = driver.find_elements_by_xpath('//*[@id="comments-speech-bubble-top"]')

In [80]:
com[0].text

'48'

In [70]:
if 'archive.nytimes' in driver.current_url:
    print(True)

True


In [71]:
match_links1['link'][400:]

400      https://www.nytimes.com/2012/11/02/nyregion/ga...
401      https://www.nytimes.com/2012/11/03/business/ec...
402      https://www.nytimes.com/2012/11/02/opinion/imm...
403      https://www.nytimes.com/2012/11/03/sports/back...
404      https://www.nytimes.com/2012/11/06/science/sur...
                               ...                        
33622    https://www.nytimes.com/2013/04/18/opinion/a-s...
33623    https://www.nytimes.com/2014/03/02/us/a-growin...
33624    https://www.nytimes.com/2015/05/17/sports/olym...
33625    https://www.nytimes.com/slideshow/2015/06/10/w...
33626    https://www.nytimes.com/2016/11/01/world/europ...
Name: link, Length: 33227, dtype: object

In [ ]:
# Sign in to NYT site from here

In [12]:
comments = []
count = 0
dump_ct = 1

for link in match_links2_diff:
    
    count +=1
    
    driver.get(link)

    com = driver.find_elements_by_xpath('//*[@id="comments-speech-bubble-top"]')
    
    if len(com) == 2:
        comments.append({'link':link, 'comments':com[1].text, 'archived':0})
    elif len(com) == 1:
        comments.append({'link':link, 'comments':com[0].text, 'archived':0})
    else:
        # check if the url was redirected to an archive. If so, there may have
        # once been comments, but we're accessing an archived version of the
        # page, so we can't see them
        if 'archive.nytimes' in driver.current_url:
            comments.append({'link':link, 'comments':-1, 'archived':1})
        else:
            comments.append({'link':link, 'comments':-1, 'archived':0})
    
    # dump to json every 100 links, but keep a running tally so the last
    # will have everything
    if count==100:
        with open(f'data/comments2_backup.json', 'w') as f:
            json.dump(comments, f)
            f.close()

        dump_ct += 1
        count = 0
        
    

In [18]:
len(comments)

3144

In [19]:
with open(f'data/comments2_backup.json', 'w') as f:
    json.dump(comments, f)
    f.close()


In [20]:
with open('data/comments2_backup.json', 'r') as f:
    comments_load = json.load(f)
    
comms = pd.DataFrame(comments_load)

In [21]:
comms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      3144 non-null   object
 1   comments  3144 non-null   object
 2   archived  3144 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 73.8+ KB


In [58]:
comms['comments'].value_counts()

-1     240
70       2
185      2
57       2
177      2
      ... 
30       1
162      1
415      1
885      1
240      1
Name: comments, Length: 149, dtype: int64